<a href="https://colab.research.google.com/github/sylviawanjiku/NLP/blob/master/Eunice_Gender_Based_Discrimination_Cleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Import Libraries

In [17]:
!pip install transformers

In [18]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import re
import nltk          #for removing stopwords
import string
from string import punctuation
from nltk.stem import SnowballStemmer
from nltk.corpus import stopwords

nltk.download('stopwords')
stop = stopwords.words('english')
nltk.download("punkt")
nltk.download("wordnet")

from wordcloud import WordCloud,STOPWORDS

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import linear_model
from sklearn.model_selection import train_test_split

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


### Loading the Dataset

In [19]:
df = pd.read_csv('GBV_data_clean_v01.csv',encoding='latin-1',error_bad_lines=False)
df.head()

b'Skipping line 4425: expected 8 fields, saw 10\nSkipping line 6651: expected 8 fields, saw 9\nSkipping line 8850: expected 8 fields, saw 10\n'


,Unnamed: 0,username,new_tweets,clean_tweets,tweets_without_stopwords,replies,retweets,likes
0,0,shaybspears,We should talk more about how this dangerous w...,we should talk more about how this dangerous w...,talk dangerous woman stalking britney sending ...,1,14,27
1,1,kaylanicole9991,"And no, getting me drunk wonÃ¢ÂÂt work. Afte...",and no getting me drunk won t work after what...,getting drunk work happened male best friend j...,1,0,0
2,2,marge78355824,ItÃ¢ÂÂs not just about seeing a random pe...,it s not just about seeing a random penis ...,seeing random penis fear every woman sexual vi...,1,0,1
3,3,explorevenango,A Franklin woman who left multiple messages in...,a franklin woman who left multiple messages in...,franklin woman left multiple messages attempt ...,0,0,0
4,4,politikscommon,I did read the details. Walker was also sue...,i did read the details walker was also sued...,read details walker also sued woman accused wa...,1,0,0


In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51057 entries, 0 to 51056
Data columns (total 8 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   Unnamed: 0                51057 non-null  int64 
 1   username                  51057 non-null  object
 2   new_tweets                51055 non-null  object
 3   clean_tweets              51031 non-null  object
 4   tweets_without_stopwords  50522 non-null  object
 5   replies                   51057 non-null  object
 6   retweets                  51057 non-null  object
 7   likes                     51056 non-null  object
dtypes: int64(1), object(7)
memory usage: 3.1+ MB


In [21]:
df.columns
df['source_file'].value_counts()

In [22]:
df = df.iloc[:, 1:]
df.head(2)

,username,new_tweets,clean_tweets,tweets_without_stopwords,replies,retweets,likes
0,shaybspears,We should talk more about how this dangerous w...,we should talk more about how this dangerous w...,talk dangerous woman stalking britney sending ...,1,14,27
1,kaylanicole9991,"And no, getting me drunk wonÃ¢ÂÂt work. Afte...",and no getting me drunk won t work after what...,getting drunk work happened male best friend j...,1,0,0


In [23]:
df.describe().T

,count,unique,top,freq
username,51057,39388,jordanmarie_23,117
new_tweets,51055,46201,,54
clean_tweets,51031,44550,,150
tweets_without_stopwords,50522,43740,memo warns attorney general jeff sessions vp p...,64
replies,51057,126,0,31362
retweets,51057,256,0,40023
likes,51056,509,0,23825


### Data Cleaning

In [24]:
df.dropna(inplace=True)
df.isna().sum()


username                    0
new_tweets                  0
clean_tweets                0
tweets_without_stopwords    0
replies                     0
retweets                    0
likes                       0
dtype: int64

In [25]:
df.duplicated().sum()

4553

In [26]:
df.drop_duplicates(inplace=True)
df.duplicated().sum()

0

### Text Preprocessing

In [27]:
tweet = df['tweet']
tweet[6] 

KeyError: ignored

In [ ]:
tweets = []
for sent in tweet:
  sent = re.sub('@[\w]+','',str(sent))
  tweets.append(sent)

In [ ]:
tweets[:6]

In [ ]:
df.columns

In [ ]:
df['new_tweets'] = tweets
# rearrange the order of the columns.
df = df[['date', 'time', 'username', 'tweet', 'new_tweets', 'replies_count', 'retweets_count','likes_count']]
df.head(2)

### Rename the columns

In [ ]:
data = df.rename(columns = {'replies_count':'replies',
                            'retweets_count':'retweets', 'likes_count':'likes'}, inplace=False)

df.columns

In [ ]:
data.columns

In [ ]:
data = data[['username', 'new_tweets', 'replies', 'retweets', 'likes']]
data.head(2)

In [ ]:
SPECIAL_TOKENS = {
    'quoted': 'quoted_item',
    'non-ascii': 'non_ascii_word',
    'undefined': 'something'
}

def cleaning(text, stem_words=True):    
    def pad_str(s):
        return ' '+s+' '
    
    if pd.isnull(text):
        return ''

    stops = set(stopwords.words("english"))
    # Clean the text, with the option to stem words.
    
    # Empty question
    
    if type(text) != str or text=='':
        return ''

    # Clean the text
    text = str(text).lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub("[%s]" % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub("\w*\d\w*", "", text)


    text = re.sub("\'s", " ", text) 
    text = re.sub(" whats ", " what is ", text, flags=re.IGNORECASE)
    text = re.sub("\'ve", " have ", text)
    text = re.sub("can't", "can not", text)
    text = re.sub("n't", " not ", text)
    text = re.sub("i'm", "i am", text, flags=re.IGNORECASE)
    text = re.sub("\'re", " are ", text)
    text = re.sub("\'d", " would ", text)
    text = re.sub("\'ll", " will ", text)
    text = re.sub("e\.g\.", " eg ", text, flags=re.IGNORECASE)
    text = re.sub("(\d+)(kK)", " \g<1>000 ", text)
    text = re.sub("e-mail", " email ", text, flags=re.IGNORECASE)
    text = re.sub("\(s\)", " ", text, flags=re.IGNORECASE)
    text = re.sub("[c-fC-F]\:\/", " disk ", text)
    
    # remove comma between numbers, i.e. 15,000 -> 15000
    
    text = re.sub('(?<=[0-9])\,(?=[0-9])', "", text)
    
    ## all numbers should separate from words, this is too aggressive
    
    def pad_number(pattern):
        matched_string = pattern.group(0)
        return pad_str(matched_string)
    text = re.sub('[0-9]+', pad_number, text)
    
    # add padding to punctuations and special chars, we still need them later
    
    text = re.sub('\$', " dollar ", text)
    text = re.sub('\%', " percent ", text)
    text = re.sub('\&', " and ", text)
    text = re.sub('amp', "", text)
    
    def pad_pattern(pattern):
       matched_string = pattern.group(0)
       return pad_str(matched_string)
    text = re.sub('[\!\?\@\^\+\*\/\,\~\|\`\=\:\;\.\#\\\]', pad_pattern, text) 
        
    text = re.sub('[^\x00-\x7F]+', pad_str(SPECIAL_TOKENS['non-ascii']), text) # replace non-ascii word with special word
    
    
    text = [c for c in text if c not in string.punctuation]
    text = [c for c in text if c != '\n'] # Removing newline
    text = ''.join(text)
    text = text.split('https')[0] # Remove links
    
    # replace the float numbers with a random number, it will be parsed as number afterward, and also been replaced with word "number"
    
    text = re.sub('[0-9]+\.[0-9]+', " 87 ", text)
    
    text = re.sub("[^a-zA-Z]",  # Search for all non-letters
                          " ",          # Replace all non-letters with spaces
                          str(text))
    
    # Remove punctuation from text
    text = ''.join([c for c in text if c not in punctuation]).lower()
       # Return a list of words
    return text

In [ ]:
# Clean the tweets.
data['clean_tweets'] = data['new_tweets'].apply(cleaning)
data.head(5)

In [ ]:
def nonasciiword (text):
  text = re.sub("nonasciiword","",text)

  return text

data['clean_tweets'] = data['clean_tweets'].apply(nonasciiword)
data.head(3)

In [ ]:
# Exclude stopwords with Python's list comprehension and pandas.DataFrame.apply.
data['tweets_without_stopwords'] = data['clean_tweets'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))

data = data[['username', 'new_tweets', 'clean_tweets', 'tweets_without_stopwords', 'replies', 'retweets','likes']]
data.head(3)

In [ ]:
tweets_df = pd.DataFrame()
tweets_df['clean_tweets'] = data['tweets_without_stopwords']
tweets_df.head(2)

### Exploratory Data Analysis

In [ ]:
# Check the above link for other installation instructions
!pip3 install sweetviz

# importing sweetviz
import sweetviz as sv

In [ ]:
#to analyze the data and create a report, simply call analyze() method passing in the dataframe as argument
report = sv.analyze(data)

#display the report as html
report.show_html('tweets.html')

#### Word Cloud

In [ ]:
# Create a word cloud
#
def cloud(data,backgroundcolor = 'white', width = 800, height = 600):
    wordcloud = WordCloud(stopwords = STOPWORDS, background_color = backgroundcolor,
                         width = width, height = height).generate(data)
    plt.figure(figsize = (12, 8))
    plt.imshow(wordcloud)
    plt.axis("off")
    
    plt.show()

In [ ]:
# display the word cloud.
#
cloud(' '.join(data['clean_tweets']))

In [ ]:
# Get Word Value Counts.
from collections import Counter
tweets_df['text_list'] = tweets_df['clean_tweets'].apply(lambda x:str(x).split())
top = Counter([item for sublist in tweets_df['text_list'] for item in sublist]) # Counts the frequency of words.
temp = pd.DataFrame(top.most_common(20)) # DataFrame of top 20
temp.columns = ["common_words", "count"]
temp.style.background_gradient(cmap="Blues")

In [ ]:
data.to_csv("GBV_data_clean_v01.csv")

In [ ]:
#saving the data
from google.colab import files
files.download('GBV_data_clean_v01.csv')

### Finding a Solution